In [1]:
import polars as pl
import cupy as cp
import os

In [2]:
lf_test = pl.scan_csv(os.path.join("dataset_Sep_9_2025", "test.csv"))
lf_train = pl.scan_csv(os.path.join("dataset_Sep_9_2025", "train.csv"))
lf_sample = pl.scan_csv(os.path.join("dataset_Sep_9_2025", "sample_submission.csv"))

In [3]:
df_test = lf_test.collect()
df_train = lf_train.collect()
df_sample = lf_sample.collect()

In [4]:
df_test.shape

(45, 2)

In [5]:
df_train.shape

(180, 8)

In [6]:
df_sample.shape

(45, 3)

In [7]:
df_train.schema

Schema([('timestamp', String),
        ('juror', String),
        ('repo_a', String),
        ('repo_b', String),
        ('parent', String),
        ('choice', Int64),
        ('multiplier', Float64),
        ('reasoning', String)])

In [8]:
df_test.schema

Schema([('repo', String), ('parent', String)])

In [9]:
df_sample.schema

Schema([('repo', String), ('parent', String), ('weight', Float64)])

In [10]:
df_train.head()

timestamp,juror,repo_a,repo_b,parent,choice,multiplier,reasoning
str,str,str,str,str,i64,f64,str
"""2025-01-24T19:03:52.628Z""","""L1Juror1""","""https://github.com/erigontech/…","""https://github.com/hyperledger…","""ethereum""",2,5.27,"""Erigon and Besu are both execu…"
"""2025-01-24T19:17:57.066Z""","""L1Juror1""","""https://github.com/web3/web3.j…","""https://github.com/paradigmxyz…","""ethereum""",1,2.22,"""Reth has a 2% market share in …"
"""2025-01-24T19:20:19.601Z""","""L1Juror1""","""https://github.com/erigontech/…","""https://github.com/ethereum/go…","""ethereum""",2,10.55,"""Erigon and Geth are both execu…"
"""2025-01-24T19:25:40.017Z""","""L1Juror1""","""https://github.com/ethereumjs/…","""https://github.com/ethers-io/e…","""ethereum""",2,4.0,"""EthereumJS is not used directl…"
"""2025-01-15T11:19:51.121Z""","""L1Juror10""","""https://github.com/erigontech/…","""https://github.com/ethereum/we…","""ethereum""",1,50.0,"""A performant and robust execut…"


In [11]:
df_sample.head()

repo,parent,weight
str,str,f64
"""https://github.com/a16z/helios""","""ethereum""",0.013638
"""https://github.com/alloy-rs/al…","""ethereum""",0.033943
"""https://github.com/apeworx/ape""","""ethereum""",0.008906
"""https://github.com/chainsafe/l…","""ethereum""",0.007414
"""https://github.com/consensys/t…","""ethereum""",0.007


In [12]:
df_test.head()

repo,parent
str,str
"""https://github.com/a16z/helios""","""ethereum"""
"""https://github.com/alloy-rs/al…","""ethereum"""
"""https://github.com/apeworx/ape""","""ethereum"""
"""https://github.com/chainsafe/l…","""ethereum"""
"""https://github.com/consensys/t…","""ethereum"""
